In [183]:
# !pip install opencv-python

In [2]:
import numpy as np
import cv2
def show_img(image, title="image"):
    if image is None:
        print("No data")
    else:
        cv2.imshow(title, image)
        cv2.waitKey()
        cv2.destroyAllWindows()
def detail_img(image, read_img):
    print(f"- Data Type: {type(image)}")
    print(f"- Data Shape: {read_img.shape} + Height x Width x Channel")
    print(f"- Maximum Intensity: {read_img.max()}")
    print(f"- Minimum Intensity: {read_img.min()}")
def normalization(image):
    image = image.astype(np.float32)
    img_norm = ((image - image.min())*(255))/(image.max()-image.min())
    img_norm = img_norm.astype(np.uint8)
    return img_norm

# 1. function을 이용한 변환
## : negative, log, power-law transformation(감마값의 범위에 따라 다양한 결과 분석, >1, <1의 각 범위에서 두가지 값 이상 설정)
## : Fig0308, Fig0309, Fig0525

In [3]:
# 이미지 주소 설정
Fig0308 = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-3  Fig0308(a)(fractured_spine) .tif"
Fig0309 = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-2 Fig0309(a)(washed_out_aerial_image).tif"
Fig0525_b = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-2  Fig0525(b)(aerial_view_turb_c_0pt0025) .tif"
Fig0525_c = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-2  Fig0525(c)(aerial_view_turb_c_0pt001).tif"
images = [Fig0308, Fig0309, Fig0525_b, Fig0525_c]
read_images = []
for image in images:
    read_images.append(cv2.imread(image, cv2.IMREAD_UNCHANGED))
for i in range(len(images)):
    detail_img(images[i], read_images[i])

- Data Type: <class 'str'>
- Data Shape: (976, 746) + Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0
- Data Type: <class 'str'>
- Data Shape: (769, 765) + Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0
- Data Type: <class 'str'>
- Data Shape: (480, 480) + Height x Width x Channel
- Maximum Intensity: 211
- Minimum Intensity: 44
- Data Type: <class 'str'>
- Data Shape: (480, 480) + Height x Width x Channel
- Maximum Intensity: 231
- Minimum Intensity: 33


## 1.1 Negative

In [4]:
def negative(image):
    return (255 - image)
negative_images = []
for image in read_images:
    negative_images.append(negative(image))
for img in negative_images:
    show_img(img)

## 1.2 Log

In [15]:
def log(image):
    c = 3
    log_image = c * np.log1p(image) * (255 / np.log(256))
    img_norm = normalization(log_image)
    return img_norm
    
log_images = []
for image in read_images:
    log_images.append(log(image))
for img in log_images:
    show_img(img)

## 1.3 Power-law transformations

In [6]:
r1 = 2.0 # 감마값 > 1
r2 = 0.5 # 감마값 < 1
def power_law_transform(image, r):
    power_img = np.power(image, r)
    img_norm = normalization(power_img)
    return img_norm
power_low_r1_images = []
power_low_r2_images = []
for image in read_images:
    power_low_r1_images.append(power_law_transform(image, r1))
    power_low_r2_images.append(power_law_transform(image, r2))
for img in power_low_r1_images:
    show_img(img)
for img in power_low_r2_images:
    show_img(img)

  # 2. Histogram Processing을 통한 대비 개선
  ## : Histogram Equalization 구현
  ## : Fig0316

In [7]:
Fig0316_1 = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-1 Fig0316(1)(top_left).jpg"
Fig0316_2 = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-1 Fig0316(2)(2nd_from_top).jpg"
Fig0316_3 = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-1 Fig0316(3)(third_from_top).jpg"
Fig0316_4 = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/HW1-1 Fig0316(4)(bottom_left).jpg"

images = [Fig0316_1, Fig0316_2, Fig0316_3, Fig0316_4]
read_images = []
for image in images:
    read_images.append(cv2.imread(image, cv2.IMREAD_UNCHANGED))
for i in range(len(images)):
    detail_img(images[i], read_images[i])
print(read_images[0].shape[0])

- Data Type: <class 'str'>
- Data Shape: (500, 500) + Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 132
- Data Type: <class 'str'>
- Data Shape: (500, 500) + Height x Width x Channel
- Maximum Intensity: 138
- Minimum Intensity: 91
- Data Type: <class 'str'>
- Data Shape: (500, 500) + Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0
- Data Type: <class 'str'>
- Data Shape: (500, 500) + Height x Width x Channel
- Maximum Intensity: 83
- Minimum Intensity: 13
500


In [8]:
def histogram_equalization(image):
    # histogram 0으로 초기화
    histogram = np.zeros(256, dtype=int)
    # histogram 계산
    for i in image:
        for j in i:
            histogram[j] +=1
    # 누적 합 histogram 초기화
    sum_hist = np.zeros(256, dtype=int)
    sum = 0
    # 누적 합 histogram 계산
    for i in range(256):
        sum += histogram[i]
        sum_hist[i] = sum
    # normalized sum histogram 초기화
    normalized_sum_hist = np.zeros(256, dtype=int)
    # normalized sum histogram 계산
    for i in range(256):
        normalized_sum = sum_hist[i] * (255.0 / (image.shape[0] * image.shape[1]))
        normalized_sum_hist[i] = np.round(normalized_sum, 0)
    # 여기까지 look-up table 완성
    transformed_image = np.zeros((image.shape[0], image.shape[1]), dtype=int)
    # 완성한 look-up table을 이용해 image transform
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            transformed_image[i][j] = normalized_sum_hist[image[i][j]]
    return np.array(transformed_image, dtype=np.uint8)

In [9]:
for image in read_images:
    show_img(histogram_equalization(image))

# 3. 컬러영상에서의 point processing
## : HSI 변환 후, Intensity에 대해 위의 point processing을 수행한 후, 결과 영상을 RGB로 변환해서 출력
## : HSI 변환은 라이브러리로 처리 가능
## : 첨부된 컬러 영상들(baboon, Lenna, peppers)

In [10]:
baboon = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/baboon.png"
lenna = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/Lenna_(test_image).Webp"
pepper = "/Users/cheonbyeongjun/Desktop/Python_File/Digital_Image_Processing/Point_Processing/Homework 1. Point processing example image/peppers.png"
images = [baboon, lenna, pepper]

read_images = []
# 컬러로 이미지 읽기
for image in images:
    read_images.append(cv2.imread(image, cv2.IMREAD_COLOR))
for i in range(len(images)):
    detail_img(images[i], read_images[i])

- Data Type: <class 'str'>
- Data Shape: (512, 512, 3) + Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0
- Data Type: <class 'str'>
- Data Shape: (440, 440, 3) + Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0
- Data Type: <class 'str'>
- Data Shape: (512, 512, 3) + Height x Width x Channel
- Maximum Intensity: 237
- Minimum Intensity: 0
512


In [11]:
# RGB -> HSV(HSI)
def convert_RGB2HSI(image):
    return cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
# HSV(HSI)에서 H, S, V 분리
def split_HSI(HSI_image):
    h, s, i = cv2.split(HSI_image)
    return h, s, i
# HSV(HSI) -> RGB
def convert_HSI2RGB(image):
    return cv2.cvtColor(image,cv2.COLOR_HSV2RGB)
# HSI image -> negative 
def negative_HSI(HSI_image):
    h, s, i = split_HSI(HSI_image)
    negative_i = negative(i)
    return cv2.merge([h, s, negative_i])
# HSI image -> log
def log_HSI(HSI_image):
    h, s, i = split_HSI(HSI_image)
    log_i = log(i)
    return cv2.merge([h, s, log_i])
# HSI image -> power-law
def power_law_HSI(HSI_image, r):
    h, s, i = split_HSI(HSI_image)
    power_law_i = power_law_transform(i, r)
    return cv2.merge([h, s, power_law_i])
def histogram_equalization_HSI(HSI_image):
    h, s, i = split_HSI(HSI_image)
    histogram_equalization_i = histogram_equalization(i)
    return cv2.merge([h, s, histogram_equalization_i])

In [12]:
# RGB image -> HSI image로 변환
HSI_images = []
for image in read_images:
    HSI_images.append(convert_RGB2HSI(image))

In [13]:
# negative
for image in HSI_images:
    negative_img = negative_HSI(image)
    back2RGB_img = convert_HSI2RGB(negative_img)
    show_img(back2RGB_img)

In [19]:
# log
for image in HSI_images:
    log_img = log_HSI(image)
    back2RGB_img = convert_HSI2RGB(log_img)
    show_img(back2RGB_img)

In [ ]:
# power-law
r1 = 2.0 # 감마값 > 1
r2 = 0.5 # 감마값 < 1
for image in HSI_images:
    power_law_img = power_law_HSI(image, r1)
    back2RGB_img = convert_HSI2RGB(power_law_img)
    show_img(back2RGB_img)
    power_law_img = power_law_HSI(image, r2)
    back2RGB_img = convert_HSI2RGB(power_law_img)
    show_img(back2RGB_img)

In [444]:
# histogram equalization
for image in HSI_images:
    histogram_equalization_img = histogram_equalization_HSI(image)
    back2RGB_img = convert_HSI2RGB(histogram_equalization_img)
    show_img(back2RGB_img)